## PROBLEM DESCRIPTION

The data we want to analyze is styles from a group of elements. In this case we had 4 styles so we want at least one machine for every one that it will be able to analyze the data an generate new data.

So the data that we have is a set of vectors that's the output of the app that generate grids. This grids ara a composition of elements so the outuput will be a vector of id. Each grid had 48 elements and each element can have a maximum of 255 elements in it. But is not necessary to had that number, it can be possible to have less. 

<h4 align="center">$[ x_1, x_2, x_3, ..., x_{48} ]$</h4> 
<h4 align="center">$x_1 = [id_1,id_2,_id_3, ...,id_{255}]$</h4>

Each element of the vector can have a different size from the others and the list of integers are the subcategories of style of the object that was placed in the grid. Each number is a different subcategory of style of the object placed previously.

So the first objective is to descompose the input data, a vector which element is a vector of different size, to a list of vectors that will have de same length, 48, and one element in each position. So the main goal is to find the grid wich elements has more IDs so we can set the size of total vectors, $y$ that we need to descompose the data. In case that one element of the vector has lower IDs that the max we gonna put a $0$ as the null value for our data.

<h4 align="center">$input = [[id_1^1,id_2^1,_id_3^1, ...,id_{255}^1], [id_1^2,id_2^2,_id_3^2, ...,id_{255}^2], ..., [id_1^48,id_2^48,_id_3^48, ...,id_{255}^48]]$</h4> 
<h4 align="center">$output = [[id_1^1, id_1^2,..., id_1^{48}],
[id_2^1, id_2^2,..., id_2^{48}],..., [0,0,...,id_y^{48}]$</h4> 


First of all we gonna implement a function that allows us to obtain the max value of the input data.

In [ ]:
import random
import numpy as np

In [ ]:
import numpy as np
def generate_data_fake(): 
    vec = [] 
    for i in range(1000): 
        tmp = [] 
        for k in range(48): 
            tmp.append([0,2]) 
        vec.append(tmp) 
    return np.asarray(vec) 

def getMaxLengthForElements(vector):
    maxLength = 0
    for i in range(len(vector)):
        tmp = vector[i]
        for j in range(len(tmp)):
            if(len(tmp[j])>maxLength):
                maxLength = len(tmp[j])
    return maxLength
                
# testVector = [[1],[1,2,3],[3,4,3,3]]
testVector = generate_data_fake()
# for i in range(500):
#     testVector[i][0:24] = 2
print(getMaxLengthForElements(testVector))

Now we got the max length that will be the number of ouput vectors that we wanna obtain from the input. Now we need to transform the input data to the output data format. For now we gonna tet with a test vector.

In [ ]:
maxLength = getMaxLengthForElements(testVector)
def transformInputVector(vector, maxLength):
    output = []
    for i in range(len(vector)):
        v = vector[i]
        for j in range(maxLength):
            tmp = []
            for k in range(len(v)):
                if(len(v[k]) < maxLength):
                    tmp.append(0)
                else:
                    tmp.append(v[k][j])
            output.append(tmp)
    return output
output_vector = transformInputVector(testVector,maxLength)

At this poing we have the data formatted ready for the input for the machine. In this case we gonna use the algorithm of the library sklearn, BernoulliRBM. Also we gonna use the implementation of One Hot Vector to transform our data to One Hot Vectors.

http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Now the first step to get the data in the machine is to convert all the data we formatted to one hot vectors. A one hot vector is a vector which is is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM
import numpy as np

In [ ]:
oneHotEncoder = OneHotEncoder(255, sparse=True).fit(output_vector)
oneHotData = oneHotEncoder.transform(output_vector)
print(oneHotData)

At this point we have 3 hot vectors which one represents the previous input data. This is a sparse matrix so is fastest to work than a simple vector and it's easy to show the results.
Then we neeed to define the parameters of the machine and fit it with de one hot vectors we had. The parameters are not specified, so we need to try whichc are the best to obtain good results

In [ ]:
RBM_Machine = BernoulliRBM(n_components=500, learning_rate=0.01, n_iter=20, random_state=0, verbose=True)
RBM_Machine.fit(oneHotData)

In [ ]:
x_visible = RBM_Machine.gibbs(oneHotData[55])
x_visible = x_visible.ravel()
final_vector = []
for k in range(len(x_visible)):
    if(x_visible[k] == True):
        final_vector.append(1)
    else:
        final_vector.append(0)
final_Data = []
for i in range(48):
    tmp = final_vector[(255*i):((i+1)*255)]
    if(1 not in tmp):
        index = 0
    else:
        index = tmp.index(1)
    final_Data.append(index)
print (final_Data)

### OWN RBM IMPLEMENTATION

In [ ]:
import sys
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from timeit import default_timer as timer
import numpy as np
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint

In [ ]:
maxLength = getMaxLengthForElements(testVector)
print(maxLength)
output_vector = transformInputVector(testVector,maxLength)
oneHotEncoder = OneHotEncoder(255, sparse=False).fit(output_vector)
oneHotData = oneHotEncoder.transform(output_vector)

In [ ]:
visible_dim = oneHotData.shape[1]
hidden_dim = 500
epochs = 100
K = 1
lr = 0.1
batch_size = 10

In [ ]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)
rbm_.W.shape, rbm_.b.shape, rbm_

In [ ]:
test_Data_Vector_Aux = np.array(oneHotData, dtype="float64")

In [ ]:
# %%time
# rbm_.fit(test_Data_Vector_Aux, 
#          method='CDK',
#          K=K,
#          lr=lr,
#          epochs=1,
#          batch_size=128,
#          plot_weights=False)

In [ ]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=1,
         batch_size=128,
         plot_weights=False)

In [ ]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)

In [ ]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=100,
         batch_size=64,
         plot_weights=False)

In [ ]:
x_hat, x_hat_p = rbm_.sample_visible_from_visible(oneHotData[0])

In [ ]:
final_vector = []
for k in range(len(x_hat)):
    if(x_hat[k] == True):
        final_vector.append(1)
    else:
        final_vector.append(0)
final_Data = []
for i in range(48):
    tmp = final_vector[(255*i):((i+1)*255)]
    if(1 not in tmp):
        index = 0
    else:
        index = tmp.index(1)
    final_Data.append(index)
print (final_Data)

### POPULAR MODERN

In [1]:
import pandas as pd
import json
import sys
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from timeit import default_timer as timer
import numpy as np
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
import random
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM

In [2]:
def getMaxLengthForElements(vector):
    maxLength = 0
    for i in range(len(vector)):
        tmp = vector[i]
        for j in range(len(tmp)):
            if(len(tmp[j])>maxLength):
                maxLength = len(tmp[j])
    return maxLength

def transformInputVector(vector, maxLength, length):
    output = []
    for i in range(length):
        v = vector[i]
        for j in range(maxLength):
            tmp = []
            length_v = len(v)
            for k in range(length_v):
                if(j >= len(v[k])):
                    tmp.append(0)
                else:
                    tmp.append(v[k][j])
            output.append(tmp)
    return output

In [6]:
vector_data = pd.read_json('./Data.json')
A = vector_data.values
data_modern = []
A = A.ravel()
length_vectors = len(A)
for i in range(A.shape[0]):
    len_v = len(A[i])
    tmp = []
    for j in range(len_v):
        tmp.append(A[i][j])
    tmp = np.asarray(tmp).ravel()
    data_modern.append(tmp)
data_modern = np.asarray(data_modern)
input_data = []
input_data.append(data_modern)
input_data = np.asarray(input_data)
max_elements = getMaxLengthForElements(input_data[0])
f_data = transformInputVector(input_data[0], max_elements, length_vectors)
for i in range(200):
    for k in range(32):
        f_data.append(f_data[k])
f_data = np.asarray(f_data)
print(f_data.shape)

(6432, 48)


In [8]:
oneHotEncoder = OneHotEncoder(262, sparse=True).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)

In [9]:
RBM_Machine = BernoulliRBM(n_components=250, learning_rate=0.01, n_iter=10, random_state=0, verbose=True)
RBM_Machine.fit(oneHotData)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -64.36, time = 56.70s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -95.83, time = 55.54s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -107.29, time = 53.69s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -113.58, time = 54.82s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -121.35, time = 53.97s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -115.21, time = 56.75s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -103.52, time = 54.22s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -94.81, time = 59.88s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -123.52, time = 65.30s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -96.16, time = 61.21s


BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=250, n_iter=10,
       random_state=0, verbose=True)

In [68]:
final_output = []
for i in range(max_elements):
    x_visible = RBM_Machine.gibbs(oneHotData[0])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
pos = 0
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

[[218, 253], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [234, 176], [237, 237, 200, 4, 200], [14, 237], [239, 14, 197, 197, 197], [214, 232, 240, 232, 14, 218, 176, 246, 234], [130, 106, 13], [183, 237, 245, 183], [10, 10], [198, 218, 237, 176], [239, 176, 239, 9], [254, 8, 45, 10, 254], [8, 183, 201], [31, 155, 240], [237, 198, 218, 4, 218], [13, 13, 219], [219, 219, 219], [224], [9, 202, 202, 9], [0], [246, 9], [220, 184, 248], [5, 237, 218], [218, 14, 219, 36]]


In [73]:
import codecs, json 
b = output_grid
json_file = "output_grid.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

In [71]:
maxLength = getMaxLengthForElements(input_data)
print(maxLength)
output_vector = transformInputVector(input_data,maxLength)
oneHotEncoder = OneHotEncoder(262, sparse=False).fit(output_vector)
oneHotData = oneHotEncoder.transform(output_vector)
test_Data_Vector_Aux = np.array(oneHotData, dtype="float64")

48


TypeError: transformInputVector() missing 1 required positional argument: 'length'

In [ ]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)

In [ ]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=50,
         batch_size=128,
         plot_weights=False)

In [ ]:
x_hat, x_hat_p = rbm_.sample_visible_from_visible(oneHotData[random.randint(0,47)])
final_vector = []
for k in range(len(x_hat)):
    if(x_hat[k] == True):
        final_vector.append(1)
    else:
        final_vector.append(0)
final_Data = []
for i in range(48):
    tmp = final_vector[(262*i):((i+1)*262)]
    if(1 not in tmp):
        index = 0
    else:
        index = tmp.index(1)
    final_Data.append(index)
print (final_Data)